# Cross-Validation und Holdout-Verfahren

Du hast nun bereits Gütekriterien für Machine Learning-Verfahren kennengelernt. In dem von dir stratifizierten Datenset kannst du z.B. die Accuracy verwenden.

Allerdings könnte genau diese Accuracy (wie auch Precision und Recall) von der Art und Weise abhängen, wie du das Datenset in Trainings- und Testdaten aufgeteilt hast. Ein gutes Modell sollte davon unabhängig sein. Deswegen wirst du jetzt versuchen, unterschiedliche Splits auszuprobieren und die Stabilität des Modells damit testen.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In diesem Teil betrachtest du nur die Klassifikation nach Autoren. Genauso kannst du das natürlich für Keywords (einzeln!) oder für die Kommentare durchführen:

## Daten für Autoren-Klassifikation vorbereiten

Diesen Teil kennst du schon:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

min_articles = min(top_authors["title"])
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

Auch zur Berechnung der Qualitätsmetriken benötigst du einen Split in Trainings- und Testdaten

## Training und Auswertung SVM

Die Aufteilung in Trainings- und Testdaten wird automatisch stratifiziert durchgeführt mit `StratifiedKFold`.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state=42)
scores = cross_val_score(clf, tfidf_vectors, adf["author"].values, scoring="accuracy")
scores

In [ ]:
cross_val_score(clf, tfidf_vectors, adf["author"].values, scoring="precision_macro")

Möchtest du mehrere Scores auf einmal ausrechnen, ist das auch möglich. Dazu musst du allerdings die Funktion `cross_validate` verwenden, die dir auch noch mehr Ergebnisse zurückliefert:

In [ ]:
from sklearn.model_selection import cross_validate
clf = SGDClassifier(random_state=42)
cv = cross_validate(clf, tfidf_vectors, y=adf["author"].values, 
                    scoring=("accuracy", "precision_weighted", "recall_weighted"))
cv

Die identischen Werte für `test_accuracy` und `test_recall_weighted` sind auf den ersten Blick erstaunlich - das liegt aber an der Konstruktion unseres "ausgeglichenen" Trainingssets.

Die Stabilität des Klassifikators erkennst du z.B. gut am Verhältnis zwischen Mittelwert und Standardabweichung, das sollte nicht zu groß sein:

In [ ]:
print(cv['test_accuracy'].std() / cv['test_accuracy'].mean())

## Vergleich der Klassifikatoren

Genau wie du im letzten Teil die Güte der Klassifikatoren ermittelt hast, kannst du nun ihre Stabilität überprüfen:

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

for clf_class in [SVC, SGDClassifier, MultinomialNB, 
                  DecisionTreeClassifier, RandomForestClassifier]:
    clf = clf_class(random_state=42) if clf_class != MultinomialNB else clf_class()
    print(clf_class.__name__)
    cv = cross_validate(clf, tfidf_vectors, y=adf["author"].values, 
                         scoring=("accuracy", "precision_weighted", "recall_weighted"))
    for m in ["accuracy", "precision_weighted", "recall_weighted"]:
        print(m, cv[f'test_{m}'].mean(), cv[f'test_{m}'].std() / cv[f'test_{m}'].mean())
    print("\n")

## Holdout-Verfahren zeigt dir die Stabilität eines Klassifikators

Mithilfe des Holdout-Verfahrens und der Kreuzvalidierung kannst du schnell unterschiedliche Konfigurationen von Trainings- und Testdaten erzeugen und damit die Stabilität deines Modells testen.

Eine Überprüfung ist zwar etwas langwierig, du solltest sie aber unbedingt durchführen. Nur so kannst du sicherstellen, dass deine Ergebnisse auch valide sind.